---
# Demo Use Case - Object Detection and Video Player (Part 2)

The raw video is now being continuously ingested into SDP.

As each video frame is ingested, the object detection model is used to detect objects such as cars, buses, and people.
The object detection model runs as a Flink job in SDP. It uses the popular YOLO (You Only Look Once) object detection model.
The Flink job uses Java bindings for TensorFlow and it uses GPUs to performance high speed deep learning inference.

We use a cluster of 6 servers, each with an NVIDIA Tesla V100 GPU. These are used in parallel by our Flink job.

See the [SDP UI](https://videodemo1.nautilus-lab-sulfur.com/).

Now the data scientist wants to view the result of the object detection model in real-time.
This is shown as a sequence of video frames with labeled boxes around detected objects.
The images from all cameras is combined into the same SDP stream.
Each image is tagged with the original camera number.

---

### Install dependencies

See [install_dependencies.ipynb](install_dependencies.ipynb).

### How to use this Notebook
1. Click *Kernel* -> *Restart Kernel and Run All Cells*.

### Import dependencies

In [ ]:
%load_ext autoreload
%autoreload 2

import grpc
import imp
import pravega.grpc_gateway as pravega
import pravega.video as video
from pravega.video import UnindexedStream, OutputStream
from matplotlib import pyplot as plt
import IPython
import os
import ipywidgets
from ipywidgets import Layout
import numpy as np
import pandas as pd
import time

imp.reload(video);

### Define Pravega stream parameters

In [ ]:
#gateway = os.environ['PRAVEGA_GRPC_GATEWAY_ADDRESS']
#gateway = '10.246.27.131:54672'
gateway = 'pravega-grpc-gateway.examples.frightful-four.eaglemonk.intranet.nautilus-platform-dev.com:80'
scope = 'examples'
#stream = 'object-detector-input-video'
#stream = 'object-detector-output-video'
#stream = 'generated-video'
stream = 'object-detector-input-grid-video'

### Initialize connection to Pravega GRPC Gateway

In [ ]:
pravega_channel = grpc.insecure_channel(gateway, options=[
        ('grpc.max_receive_message_length', 9*1024*1024),
    ])
pravega_client = pravega.grpc.PravegaGatewayStub(pravega_channel)

### View the size of the video stream

In [ ]:
unindexed_stream = UnindexedStream(pravega_client, scope, stream)

In [ ]:
#unindexed_stream.truncate_stream()

In [ ]:
stream_info = unindexed_stream.get_stream_info()
stream_info

In [ ]:
stream_size_MB = (list(stream_info.tail_stream_cut.cut.values())[0] - list(stream_info.head_stream_cut.cut.values())[0]) * 1e-6
stream_size_MB

In [ ]:
# def play_video(self, from_stream_cut=None, to_stream_cut=None, show_frame_interval=1,
#                cameras=None, tz='America/Los_Angeles', strftime='%Y-%m-%d %I:%M:%S.%f %p %z', buffer_sec=0.0):
#     """Play a video from a Pravega stream using a Jupyter notebook."""
#     read_events = self.read_video_frames(from_stream_cut, to_stream_cut, cameras=cameras)
#     header_output = ipywidgets.Output()
#     body_output = ipywidgets.Output()
#     IPython.display.display(header_output, body_output)
#     for i, video_frame in enumerate(read_events):
#         if i % show_frame_interval == 0:
#             timestamp = video_frame['timestamp']
#             sleep_sec = buffer_sec - (pd.Timestamp.utcnow() - timestamp).total_seconds()
#             if sleep_sec > 0.0:
#                 time.sleep(sleep_sec)
#             dt = str(pd.Timestamp.utcnow() - timestamp)
#             timestamp_value = '%s  (%s)' % (timestamp, timestamp.astimezone(tz).strftime(strftime))
#             timestamp_widget = ipywidgets.Text(description='Timestamp', disabled=True, value=timestamp_value, layout=Layout(width='50%'))
#             dt_widget = ipywidgets.Text(description='Age', disabled=True, value=dt, layout=Layout(width='25%'))
#             camera_widget = ipywidgets.Text(description='Camera', disabled=True, value=str(video_frame['camera']), layout=Layout(width='15%'))    
#             header_widget = ipywidgets.HBox([camera_widget, timestamp_widget, dt_widget])
#             img_widget = IPython.display.Image(data=video_frame['data'])
#             with header_output:
#                 IPython.display.clear_output(wait=True)
#                 IPython.display.display(header_widget)
#             with body_output:
#                 IPython.display.clear_output(wait=True)
#                 IPython.display.display(img_widget)

### Play video stream from the current time

In [ ]:
stream_info = unindexed_stream.get_stream_info()
from_stream_cut = stream_info.tail_stream_cut
to_stream_cut = None
unindexed_stream.play_video(from_stream_cut, to_stream_cut, cameras=[0,1000], show_frame_interval=1)

# Play video stream from the earliest time

In [ ]:
play_video(unindexed_stream, cameras=None)

# Play video stream from the earliest time and stop at current time

In [ ]:
stream_info = unindexed_stream.get_stream_info()
from_stream_cut = stream_info.head_stream_cut
to_stream_cut = stream_info.tail_stream_cut
play_video(unindexed_stream, from_stream_cut, to_stream_cut, cameras=[0])

# Play video stream from a stream cut

In [ ]:
from_stream_cut_text = 'H4sIAAAAAAAAADOwSq1IzC3ISS3Wz0/KSk0u0U1JLQFS+UW6+aUlBaUlumWZKan5VgZAaGhoamJqYGxpYQIA+ZiLQTYAAAA='
from_stream_cut = pravega.pb.StreamCut(text=from_stream_cut_text)
to_stream_cut = None
play_video(unindexed_stream, from_stream_cut, to_stream_cut, cameras=[0])

# Playground